# Data Cleaning and Data Wrangling

## Data

Dataset contains socio-demographic and firmographic features about 2.240 
customers.

|Feature |Description|
|--:|---|
|AcceptedCmp1| 1 if customer accepted the offer in the 1st campaign, 0 otherwise|
|AcceptedCmp2| 1 if customer accepted the offer in the 2nd campaign, 0 otherwise|
|AcceptedCmp3| 1 if customer accepted the offer in the 3rd campaign, 0 otherwise|
|AcceptedCmp4| 1 if customer accepted the offer in the 4th campaign, 0 otherwise|
|AcceptedCmp5| 1 if customer accepted the offer in the 5th campaign, 0 otherwise|
|Response (target)| 1 if customer accepted the offer in the last campaign, 0 otherwise|
|Complain| 1 if customer complained in the last 2 years|
|DtCustomer| date of customer's enrollment with the company|
|Education| customer's level of education|
|Marital| customer's marital status|
|Kidhome| number of small children in customer's household|
|Teenhome |number of teenagers in customer's household|
|Income| customer's yearly household income|
|MntFishProducts| amount spent on fish products in the last 2 years|
|MntMeatProducts| amount spent on meat products in the last 2 years|
|MntFruits| amount spent on fruits products in the last 2 years|
|MntSweetProducts| amount spent on sweet products in the last 2 years|
|MntWines| amount spent on wines products in the last 2 years|
|MntGoldProds| amount spent on gold products in the last 2 years|
|NumDealsPurchases| number of purchases made with discount|
|NunCatalogPurchases| number of purchases made using catalog|
|NunStorePurchases| number of purchases made directly in stores|
|NumWebPurchases| number of purchases made through company's web site|
|NumWebVisitsMonth| number of visits to company's web site in the last month|
|Recency|number of days since the last purchase|
|Z_Revenue|revenue from the new gadget|
|Z_CostContact|cost of contact for the sixth campaign|

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from scipy import stats
from scipy.stats import kstest

import warnings
warnings.filterwarnings("ignore")

## Data Exploration

In [2]:
# Storing path
path = Path("../data/ifood_customers.csv")

# Read CSV with pandas
data = pd.read_csv(path)

# showing a sample of the dataset
data.sample(8)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
1436,8588,1961,Graduation,Married,60544.0,1,1,2012-08-25,92,201,...,6,0,0,0,0,0,0,3,11,0
2228,8720,1978,2n Cycle,Together,NaN,0,0,2012-08-12,53,32,...,0,0,1,0,0,0,0,3,11,0
1243,7718,1947,Master,Together,66000.0,0,0,2014-04-20,36,244,...,1,0,0,0,0,0,0,3,11,0
1319,8749,1984,Graduation,Together,37235.0,1,0,2014-02-01,68,20,...,4,0,0,0,0,0,0,3,11,0
2017,10598,1967,Graduation,Together,27943.0,1,1,2013-04-15,89,12,...,8,0,0,0,0,0,0,3,11,0
1014,5313,1971,Master,Married,38196.0,1,1,2014-04-18,20,30,...,5,0,0,0,0,0,0,3,11,0
1303,3099,1970,Graduation,Divorced,44267.0,1,1,2013-02-25,48,183,...,9,0,0,0,0,0,0,3,11,0
296,2874,1988,2n Cycle,Divorced,35388.0,1,0,2013-03-07,20,6,...,7,0,0,0,0,0,0,3,11,0


In [3]:
# Exploring the columns and dtype

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [4]:
# Exploring unique values in some columns

print("\n'Education' Column Values:\n", "\t",
      data["Education"].unique(),
      sep=''
      )

print("\n'Marital_Status' Column Values:\n", "\t",
      data["Marital_Status"].unique(),
      sep=''
      )

print("\n'Dt_Customer' Column sample:\n",
      data["Dt_Customer"].sample(3), "\n", "\t",
      sep=''
      )



'Education' Column Values:
	['Graduation' 'PhD' 'Master' 'Basic' '2n Cycle']

'Marital_Status' Column Values:
	['Single' 'Together' 'Married' 'Divorced' 'Widow' 'Alone' 'Absurd' 'YOLO']

'Dt_Customer' Column sample:
223     2013-01-05
1346    2013-02-10
1329    2013-11-17
Name: Dt_Customer, dtype: object
	


## Creating columns

In [5]:
# Stabilizing dtype 'category' to Education and Marital_Status columns
# Also Dt_Customer to Datetime dtype and Income to 'Int64'

data["Income"] = data["Income"].astype("Int64")
data["Education"] = data["Education"].astype("category")
data["Marital_Status"] = data["Marital_Status"].astype("category")
data["Dt_Customer"] = pd.to_datetime(data["Dt_Customer"], format="%Y-%m-%d")

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   2240 non-null   int64         
 1   Year_Birth           2240 non-null   int64         
 2   Education            2240 non-null   category      
 3   Marital_Status       2240 non-null   category      
 4   Income               2216 non-null   Int64         
 5   Kidhome              2240 non-null   int64         
 6   Teenhome             2240 non-null   int64         
 7   Dt_Customer          2240 non-null   datetime64[ns]
 8   Recency              2240 non-null   int64         
 9   MntWines             2240 non-null   int64         
 10  MntFruits            2240 non-null   int64         
 11  MntMeatProducts      2240 non-null   int64         
 12  MntFishProducts      2240 non-null   int64         
 13  MntSweetProducts     2240 non-nul

In [6]:
# Creating new columns: customer year old and days since becoming customer

today = pd.to_datetime(datetime.today().strftime('%Y-%m-%d'))

data["Year_Old"] = (today.year - data["Year_Birth"])
data["CustomerFor"] = (today - data["Dt_Customer"])

data[["Year_Old", "CustomerFor"]].sample(5)

,Year_Old,CustomerFor
778,69,3854 days
1934,73,4108 days
165,58,3889 days
1406,73,4105 days
206,36,3864 days


In [7]:
# Reordering columns

pop_column = data.pop("Dt_Customer")
data.insert(2, "Dt_Customer", pop_column)

last_columns = data.columns[-2:]
first_columns = data.columns[:2]
middle_columns = data.columns[2:-2]
new_order = list(first_columns) + list(last_columns) + list(middle_columns)

data = data[new_order]

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   ID                   2240 non-null   int64          
 1   Year_Birth           2240 non-null   int64          
 2   Year_Old             2240 non-null   int64          
 3   CustomerFor          2240 non-null   timedelta64[ns]
 4   Dt_Customer          2240 non-null   datetime64[ns] 
 5   Education            2240 non-null   category       
 6   Marital_Status       2240 non-null   category       
 7   Income               2216 non-null   Int64          
 8   Kidhome              2240 non-null   int64          
 9   Teenhome             2240 non-null   int64          
 10  Recency              2240 non-null   int64          
 11  MntWines             2240 non-null   int64          
 12  MntFruits            2240 non-null   int64          
 13  MntMeatProducts   

In [8]:
# Checking if the customer bought in the last month

data["PurchaseLastMonth"] = (data["Recency"] < 30)
data["PurchaseLastMonth"] = data["PurchaseLastMonth"].replace({True:1, 
                                                               False:0})

print(data["PurchaseLastMonth"].sample(5))

764     0
1574    0
21      0
1152    0
1864    1
Name: PurchaseLastMonth, dtype: int64


In [9]:
# Calculating total amount spent per customer

MntSpentTotal_sum = ["MntFishProducts", "MntFruits", "MntGoldProds", 
                     "MntMeatProducts", "MntSweetProducts", "MntWines"]
data["MntSpentTotal"] = data[MntSpentTotal_sum].sum(axis=1)

print(data["MntSpentTotal"].sample(5))

182     1033
105       29
956      475
1722    1991
1684     304
Name: MntSpentTotal, dtype: int64


In [10]:
# How many campaigns the customer accepted

AcceptedCmpTotal_sum = ["AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3", 
                        "AcceptedCmp4", "AcceptedCmp5", "Response"]
data["AcceptedCmpTotal"] = data[AcceptedCmpTotal_sum].sum(axis=1)

data["AcceptedCmpTotal"].sample(5)

582     0
2052    0
2153    1
22      1
1102    0
Name: AcceptedCmpTotal, dtype: int64

In [11]:
# How many children (Kids and teenagers) the customer has at home

ChildrenHome_sum = ["Kidhome", "Teenhome"]
data["ChildrenHome"] = data[ChildrenHome_sum].sum(axis="columns")

data["ChildrenHome"].sample(5)

2201    1
1914    1
1474    1
1193    1
710     1
Name: ChildrenHome, dtype: int64

### Columns reordered

In [12]:
# Reordering columns

pop_column = data.pop("AcceptedCmpTotal")
data.insert(27, "AcceptedCmpTotal", pop_column)

pop_column = data.pop("PurchaseLastMonth")
data.insert(17, "PurchaseLastMonth", pop_column)

pop_column = data.pop("MntSpentTotal")
data.insert(11, "MntSpentTotal", pop_column)

pop_column = data.pop("ChildrenHome")
data.insert(10, "ChildrenHome", pop_column)

pop_column = data.pop("AcceptedCmp2")
data.insert(25, "AcceptedCmp2", pop_column)

pop_column = data.pop("AcceptedCmp1")
data.insert(25, "AcceptedCmp1", pop_column)

pop_column = data.pop("Response")
data.insert(30, "Response", pop_column)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   ID                   2240 non-null   int64          
 1   Year_Birth           2240 non-null   int64          
 2   Year_Old             2240 non-null   int64          
 3   CustomerFor          2240 non-null   timedelta64[ns]
 4   Dt_Customer          2240 non-null   datetime64[ns] 
 5   Education            2240 non-null   category       
 6   Marital_Status       2240 non-null   category       
 7   Income               2216 non-null   Int64          
 8   Kidhome              2240 non-null   int64          
 9   Teenhome             2240 non-null   int64          
 10  ChildrenHome         2240 non-null   int64          
 11  Recency              2240 non-null   int64          
 12  MntSpentTotal        2240 non-null   int64          
 13  MntWines          

## Data Cleaning

In [13]:
# Missing Values

row_nan = data[data.isna().any(axis=1)]
print("Customers with missing values: ", len(row_nan))

data.drop(row_nan.index, inplace=True)

Customers with missing values:  24


In [14]:
# Identifying logically incoherent customers and dropping from the dataframe

marital_filt = data[data["Marital_Status"].isin(['Alone', 'Absurd', 'YOLO'])]
print("Customers with a logically incoherent Marital Status: ", 
      len(marital_filt))

data.drop(marital_filt.index, inplace=True)

Customers with a logically incoherent Marital Status:  7


### Kolmogorov-Smirnov test

In [15]:
# KS-Test on 'Year_Old' column
ks_result = kstest(data["Year_Old"], stats.norm.cdf, 
                   args=(data["Year_Old"].mean(), data["Year_Old"].std()))

print(f"Test statistic: {ks_result.statistic:.4f}")
print(f"p Value: {ks_result.pvalue:.4f}")

Test statistic: 0.0590
p Value: 0.0000


In [16]:
# KS-Test on 'Year_Old' column
ks_result = kstest(data["Income"], stats.norm.cdf, 
                   args=(data["Income"].mean(), data["Income"].std()))

print(f"Test statistic: {ks_result.statistic:.4f}")
print(f"p Value: {ks_result.pvalue:.4f}")

Test statistic: 0.0542
p Value: 0.0000


### Obtaining outliers

In [17]:
# Quartiles and IQR
quartiles = data["Year_Old"].quantile([0.25, 0.75])
iqr = quartiles[0.75] - quartiles[0.25]

# Identify Outliers
lower_bound = quartiles[0.25] - 1.5 * iqr
upper_bound = quartiles[0.75] + 1.5 * iqr

# Filtering
Year_Old_outliers = data[(data["Year_Old"] < lower_bound) | 
                (data["Year_Old"] > upper_bound)]

print("IQR: ", iqr)
print("outliers: ")
print(Year_Old_outliers[["ID", "Year_Old", "Income", "CustomerFor", 
                "Marital_Status", "MntSpentTotal"]])

# Extracting Outliers from the Dataset
data.drop(Year_Old_outliers.index, inplace=True)

IQR:  18.0
outliers: 
        ID  Year_Old  Income CustomerFor Marital_Status  MntSpentTotal
192   7829       124   36640   3937 days       Divorced             65
239  11004       131   60182   3704 days         Single             22
339   1150       125   83532   3937 days       Together           1853


In [18]:
# Quartiles and IQR
quartiles = data["Income"].quantile([0.25, 0.75])
iqr = quartiles[0.75] - quartiles[0.25]

# Identify Outliers
lower_bound = quartiles[0.25] - 1.5 * iqr
upper_bound = quartiles[0.75] + 1.5 * iqr

# Filtering
income_outliers = data[(data["Income"] < lower_bound) | 
                (data["Income"] > upper_bound)]

print("IQR: ", iqr)
print("outliers: ")
print(income_outliers[["ID", "Year_Old", "Income", "CustomerFor", 
                "Marital_Status", "MntSpentTotal"]]\
                    .sort_values("Income", ascending=False))

# Extracting logically incoherent Outlier from the Dataset
data.drop(2233, inplace=True)

IQR:  33383.5
outliers: 
         ID  Year_Old  Income CustomerFor Marital_Status  MntSpentTotal
2233   9432        47  666666   4053 days       Together             62
617    1503        48  162397   4052 days       Together            107
687    1501        42  160803   4355 days        Married           1717
1300   5336        53  157733   4051 days       Together             59
164    8475        51  157243   3781 days        Married           1608
1653   4931        47  157146   4087 days       Together           1730
2132  11181        75  156924   3965 days        Married              8
655    5555        49  153924   3803 days       Divorced              6


## New Categorical columns

In [19]:
# Age binning categories

print(data["Year_Old"].describe()[["min", "mean", "max"]])

bins = [25, 35, 45, 55, 65, 75, 2000]
labels = ["25_34", "35_44", "45_54", "55_64", "65_74", "75_above"]

data["Age_cat"] = pd.cut(data["Year_Old"], bins, labels=labels, right=False)

print("\n", data[["ID", "Year_Old", "Age_cat", "Income"]].sample(5), sep='')

min     28.000000
mean    55.101134
max     84.000000
Name: Year_Old, dtype: float64

        ID  Year_Old Age_cat  Income
638   1907        74   65_74   63120
1881  8341        55   55_64   30396
1502  3340        47   45_54   42014
1044  6287        38   35_44   34728
771   1915        73   65_74   78939


In [20]:
# Income binning categories

print(data["Income"].describe()[["min", "mean", "max"]])

labels = [f"D{i+1}" for i in np.arange(0,10)]

data["Income_cat"] = pd.cut(data["Income"], 10, precision=0, labels=labels)

print("\n", data[["ID", "Age_cat", "Income", "Income_cat"]].sample(5), sep='')


min          1730.0
mean    51954.61542
max        162397.0
Name: Income, dtype: Float64

        ID Age_cat  Income Income_cat
326   5835   45_54   14849         D1
539    304   35_44   22944         D2
927   3139   35_44   74116         D5
879   3270   45_54   17117         D1
2161   902   35_44   62994         D4


In [21]:
# Total amount spent binning categories

print(data["MntSpentTotal"].describe()[["min", "mean", "max"]])

data["MntTotal_cat"], intervals = pd.cut(data["MntSpentTotal"], 6, 
                                         precision=0, retbins=True)

# Exploring bins:
# print("\n", data["MntTotal_cat"].value_counts().sort_index(), sep='')

# Creating new, more descriptive bins
temp, first_int = pd.cut(np.arange(2, 426), 5, retbins=True)
bins = list(first_int) + list(intervals[2:])

data["MntTotal_cat"], intervals = pd.cut(data["MntSpentTotal"], bins, 
                                         precision=0, right=False, 
                                         retbins=True)

print("\n", data[["ID", "Age_cat", 
                  "Income", "MntTotal_cat", 
                  "MntSpentTotal"]].sample(5), sep='')


min        5.000000
mean     607.380499
max     2525.000000
Name: MntSpentTotal, dtype: float64

        ID Age_cat  Income      MntTotal_cat  MntSpentTotal
87    4452   65_74   50388    [340.0, 425.0)            372
1566  7476   55_64   63972  [1265.0, 1685.0)           1269
63    6518   65_74   67680    [425.0, 845.0)            606
2169  4548   35_44   41967       [2.0, 87.0)             54
167   3712   65_74   52332    [256.0, 340.0)            259


In [22]:
# Recency binning categories

print(data["Recency"].describe()[["min", "mean", "max"]])

labels = ["0_24", "25_49", "50_74", "75_99"]

data["Recency_cat"] = pd.cut(data["Recency"], 4, precision=0, labels=labels)

print("\n", data[["ID", "Age_cat", "Recency", "Recency_cat"]].sample(5), 
      sep='')


min      0.000000
mean    49.082993
max     99.000000
Name: Recency, dtype: float64

         ID   Age_cat  Recency Recency_cat
1512   9451     55_64       92       75_99
1367   3231     45_54       97       75_99
872    3623     45_54       55       50_74
1019   4436     45_54       27       25_49
1634  10906  75_above       52       50_74


## Saving new Dataset

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2205 entries, 0 to 2239
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   ID                   2205 non-null   int64          
 1   Year_Birth           2205 non-null   int64          
 2   Year_Old             2205 non-null   int64          
 3   CustomerFor          2205 non-null   timedelta64[ns]
 4   Dt_Customer          2205 non-null   datetime64[ns] 
 5   Education            2205 non-null   category       
 6   Marital_Status       2205 non-null   category       
 7   Income               2205 non-null   Int64          
 8   Kidhome              2205 non-null   int64          
 9   Teenhome             2205 non-null   int64          
 10  ChildrenHome         2205 non-null   int64          
 11  Recency              2205 non-null   int64          
 12  MntSpentTotal        2205 non-null   int64          
 13  MntWines             22

In [25]:
data.to_csv("../data/ifood_customers_cleaned.csv")